<a href="https://colab.research.google.com/github/encoras/Artificial-Intelligence-Group/blob/master/opencv_motion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Approach to Moving Object Detection

https://learnopencv.com/moving-object-detection-with-opencv/

https://www.youtube.com/watch?v=O3b8lVF93jU


https://www.youtube.com/watch?v=1gi5qn1khVk

In [173]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

#or
#inputt = 'https://raw.githubusercontent.com/encoras/Introduction-to-OpenCV/master/Camera3.mp4'
#capture = cv2.VideoCapture(inputt)

inputt = '/content/drive/MyDrive/Colab Notebooks/Camera3.mp4'

capture = cv2.VideoCapture(cv2.samples.findFileOrKeep(inputt))
frame_width = int(capture.get(3))
frame_height = int(capture.get(4))
out = cv2.VideoWriter('DiFF_Background_Subtraction_Tutorial_frame_output.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30, (frame_width,frame_height))

In [ ]:
ret, frame1 = capture.read()
if not ret:
    print("Cannot read first frame")
    # exit or handle error

gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
gray1 = cv2.GaussianBlur(gray1, (5, 5), 0)

frame_count = 0

while True:
    ret, frame2 = capture.read()
    if not ret:
        print("End of video / camera error")
        break

    frame_count += 1

    # Convert to grayscale
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.GaussianBlur(gray2, (5, 5), 0)

    # Frame differencing
    diff = cv2.absdiff(gray1, gray2)

    # Threshold + noise reduction
    _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=2)          # fill small gaps

    # Find moving objects
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Make a copy to draw on (clean original)
    display_frame = frame2.copy()

    for contour in contours:
        if cv2.contourArea(contour) < 200:   # adjust this threshold
            continue

        (x, y, w, h) = cv2.boundingRect(contour)

        # Option A: Draw green rectangle (most common for motion detection)
        cv2.rectangle(display_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Option B: If you really want to "white out" the moving region
        # cv2.rectangle(display_frame, (x, y), (x+w, y+h), (255, 255, 255), -1)

        # Optional: add area text
        # cv2.putText(display_frame, f"{cv2.contourArea(contour):.0f}",
        #             (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    if frame_count < 5:
      cv2_imshow(display_frame)   # Colab
      print(f"Showing frame #{frame_count}")
    # cv2.imshow("Motion", display_frame)   # normal python
    out.write(display_frame)
    gray1 = gray2


capture.release()
out.release()

OpenCV tools


In [ ]:
#Reset camera frame pointer to 0
capture = cv2.VideoCapture(cv2.samples.findFileOrKeep(inputt))
frame_width = int(capture.get(3))
frame_height = int(capture.get(4))

algo = 'MOG2'

if algo == 'MOG2':
    backSub = cv2.createBackgroundSubtractorMOG2(history=50, varThreshold=16, detectShadows=False)
    backSub.setShadowThreshold(0.5)
    out = cv2.VideoWriter('MOG_Background_Subtraction_Tutorial_frame_output.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30, (frame_width,frame_height))
else:
    backSub = cv2.createBackgroundSubtractorKNN()
    out = cv2.VideoWriter('KNN_Background_Subtraction_Tutorial_frame_output.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30, (frame_width,frame_height))



In [ ]:
def get_motion_mask(fg_mask, min_thresh=0, kernel=np.array((9,9), dtype=np.uint8)):
    """ Obtains image mask
        Inputs:
            fg_mask - foreground mask
            kernel - kernel for Morphological Operations
        Outputs:
            mask - Thresholded mask for moving pixels
        """
    _, thresh = cv2.threshold(fg_mask,min_thresh,255,cv2.THRESH_BINARY)
    motion_mask = cv2.medianBlur(thresh, 5)

    # morphological operations
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    return motion_mask

In [ ]:
frame_count = 0
while True:
    ret, frame = capture.read()
    if not ret:
        print("End of video")
        break
    frame_count += 1
    # Apply background subtractor
    fgMask = backSub.apply(frame)

    # Get cleaned motion mask
    mask = get_motion_mask(fgMask)

    # Find contours on the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw on COLOR copy of original frame (key fix: was drawing on grayscale mask before)
    display_frame = frame.copy()  # Use frame.copy() instead of mask.copy()

    for contour in contours:
        if cv2.contourArea(contour) < 200:  # Adjust threshold as needed
            continue

        (x, y, w, h) = cv2.boundingRect(contour)

        # Draw green rectangle on color frame
        cv2.rectangle(display_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Optional: White out the region (uncomment if wanted)
        # cv2.rectangle(display_frame, (x, y), (x + w, y + h), (255, 255, 255), -1)

        # Optional: Add contour area text
        # cv2.putText(display_frame, f"{cv2.contourArea(contour):.0f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Display the result
    if frame_count < 5:
      cv2_imshow(display_frame)   # Colab
      print(f"Showing frame #{frame_count}")

    out.write(display_frame)

# Cleanup
capture.release()
out.release()





Cleaned rectangles


In [ ]:
#Reset camera frame pointer to 0
capture = cv2.VideoCapture(cv2.samples.findFileOrKeep(inputt))
frame_width = int(capture.get(3))
frame_height = int(capture.get(4))

algo = 'MOG2'

if algo == 'MOG2':
    backSub = cv2.createBackgroundSubtractorMOG2(history=50, varThreshold=16, detectShadows=False)
    backSub.setShadowThreshold(0.5)
    out = cv2.VideoWriter('c_MOG_Background_Subtraction_Tutorial_frame_output.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30, (frame_width,frame_height))
else:
    backSub = cv2.createBackgroundSubtractorKNN()
    out = cv2.VideoWriter('c_KNN_Background_Subtraction_Tutorial_frame_output.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30, (frame_width,frame_height))


In [ ]:
def filter_nested_boxes(boxes, iou_threshold=0.7):
    """
    boxes: list of [x, y, w, h]
    Returns: list of kept boxes (non-nested or outer ones)
    """
    if not boxes:
        return []

    # Sort by area descending → bigger boxes first
    indexed_boxes = sorted(enumerate(boxes), key=lambda ib: ib[1][2] * ib[1][3], reverse=True)
    kept_indices = set()

    for i, (idx, box_a) in enumerate(indexed_boxes):
        keep = True
        xa, ya, wa, ha = box_a
        area_a = wa * ha

        for j in range(i):  # only check against already processed (larger) boxes
            _, box_b = indexed_boxes[j]
            xb, yb, wb, hb = box_b

            # intersection
            xi = max(xa, xb)
            yi = max(ya, yb)
            wi = max(0, min(xa+wa, xb+wb) - xi)
            hi = max(0, min(ya+ha, yb+hb) - yi)
            inter = wi * hi

            # IoU = inter / area_a  (we care if smaller is mostly inside bigger)
            if inter / area_a > iou_threshold:
                keep = False
                break

        if keep:
            kept_indices.add(idx)

    # Return original order or sorted — your choice
    return [boxes[i] for i in sorted(kept_indices)]

In [ ]:
frame_count = 0
while True:
    ret, frame = capture.read()
    if not ret:
        print("End of video")
        break
    frame_count += 1
    # Apply background subtractor
    fgMask = backSub.apply(frame)

    # Get cleaned motion mask
    mask = get_motion_mask(fgMask)

    # Find contours on the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw on COLOR copy of original frame (key fix: was drawing on grayscale mask before)
    display_frame = frame.copy()  # Use frame.copy() instead of mask.copy()

    # Step 1: collect all bounding boxes as [x, y, w, h]
    rectangles = []
    for contour in contours:
        if cv2.contourArea(contour) < 200:          # your area filter
            continue
        x, y, w, h = cv2.boundingRect(contour)
        rectangles.append([x, y, w, h])

    if rectangles:
        rect_array = np.array(rectangles + rectangles, dtype=np.int32)  # duplicate!
        grouped, weights = cv2.groupRectangles(rect_array, groupThreshold=1, eps=0.35)

        if isinstance(grouped, np.ndarray) and len(grouped) > 0:
            grouped_rects = grouped.tolist()
        else:
            grouped_rects = rectangles  # fallback
    else:
        grouped_rects = []

    # Step 3: draw only the grouped (cleaned) rectangles
    display_frame = frame.copy()




    for rect in grouped_rects:
        x, y, w, h = rect
        cv2.rectangle(display_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the result
    if frame_count < 5:
      cv2_imshow(display_frame)   # Colab
      print(f"Showing frame #{frame_count}")

    out.write(display_frame)

# Cleanup
capture.release()
out.release()